<div style="background-color: Purple; padding: 10px;">
    <h2> Conexão com Banco de Dados no PostgreSQL
</div>

In [1]:
import pandas as pd 
from sqlalchemy import create_engine

USER = "postgres"
SENHA = "postgres"
HOST = "localhost"
DATABASE = "black"

engine = create_engine(f"postgresql+psycopg2://{USER}:{SENHA}@{HOST}/{DATABASE}")

In [23]:
# Importando Bibliotecas

import pandas as pd 
from sqlalchemy import create_engine

In [25]:
# Configuração de Conexão com o Banco de Dados PostgreSQL

USER = "postgres"   # Usuário cadastrado
SENHA = "postgres"  # Senha
HOST = "localhost"  # Host do servidor PostgreSQL
DATABASE = "black"  # Nome do banco de dados

engine = create_engine(f"postgresql+psycopg2://{USER}:{SENHA}@{HOST}/{DATABASE}")

---

```
CREATE TABLE categorias 
( 
 id_categoria INTEGER PRIMARY KEY,
 nome_categoria TEXT
);

CREATE TABLE clientes 
( 
 id_cliente INTEGER PRIMARY KEY,
 nome_cliente TEXT,
 idade INTEGER,
 endereco TEXT
);

CREATE TABLE fornecedores 
( 
 id_fornecedor INTEGER PRIMARY KEY,
 nome TEXT,
 contato TEXT 
);

CREATE TABLE marcas 
( 
 id_marca TEXT PRIMARY KEY,
 nome TEXT
);

CREATE TABLE produtos 
( 
 id_produto INTEGER PRIMARY KEY,
 nome_produto TEXT,
 preco REAL,
 categoria_id INTEGER,
 marca_id TEXT,
 fornecedor_id INTEGER,
 data_estoque DATE,
 status TEXT,
 FOREIGN KEY(categoria_id) REFERENCES categorias (id_categoria),
 FOREIGN KEY(marca_id) REFERENCES marcas (id_marca),
 FOREIGN KEY(fornecedor_id) REFERENCES fornecedores (id_fornecedor)
);

CREATE TABLE vendas 
( 
 id_venda INTEGER PRIMARY KEY,
 data_venda DATE,
 total_venda REAL,
 cliente_id INTEGER,
 FOREIGN KEY(cliente_id) REFERENCES clientes (id_cliente)
);

 CREATE TABLE itens_venda 
( 
 venda_id INTEGER,
 produto_id INTEGER,
 PRIMARY KEY (venda_id, produto_id),
 FOREIGN KEY(venda_id) REFERENCES vendas (id_venda),
 FOREIGN KEY(produto_id) REFERENCES produtos (id_produto)
);
```

### Ferramentas:
- Git 
- PandaSQL 
- PostgreSQL 
- pgAdmin4

### Referências:
- https://pandas.pydata.org
- https://www.postgresql.org
- https://pypi.org/project/pandasql/
- https://pypi.org/project/psycopg2/
- https://learnsql.com/blog/how-to-import-csv-to-postgresql/

---

<div style="background-color: Purple; padding: 10px;">
    <h2> Exploração Inicial dos Dados
</div>

- Tabela ``categorias``;

In [11]:
query = """
SELECT * FROM categorias
"""

df = pd.read_sql_query(query, con = engine)
print(df.shape)
df

(5, 2)


,id_categoria,nome_categoria
0,1,Eletrônicos
1,2,Vestuário
2,3,Alimentos
3,4,Livros
4,5,Esportes


Essa é uma tabela dimensão que armazena atributos das categorias de produtos desse negócio. O `id_categoria`, que é a `PRIMARY-KEY`, é o identificador único para cada categoria distinta presente, cujos nomes são representados por  `nome_categoria`.

Com o `df.shape` sabemos que essa tabela tem 5 linhas e 2 colunas.

Observamos que há somente 5 categorias: Eletrônicos, Vestuário, Alimentos, Livros e Esportes.

Utilizando o comando `DataFrame.info()` da biblioteca `pandas`, verificamos os tipos de dados de cada coluna da tabela convertida em dataframe.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id_categoria    5 non-null      int64 
 1   nome_categoria  5 non-null      object
dtypes: int64(1), object(1)
memory usage: 212.0+ bytes


Também podemos usar o comando `DataFrame.isnull().sum()` para verificar se temos dados faltantes. 

Nessa tabela, não há dados faltantes.

In [12]:
df.isnull().sum()

id_categoria      0
nome_categoria    0
dtype: int64

---

- Tabela ``clientes``;

In [13]:
query = """
SELECT * FROM clientes
"""

df = pd.read_sql_query(query, con = engine)
print(df.shape)
df.head()

(10000, 4)


,id_cliente,nome_cliente,idade,endereco
0,1,Carla,50,PR
1,2,Felipe,36,RJ
2,3,Elena,42,DF
3,4,Alice,29,PA
4,5,Alice,44,MA


A tabela clientes contém os atributos de nome, idade e endereço de cada cliente desse negócio, sendo `id_cliente` a `PRIMARY KEY`.

Aqui, temos 10.000 linhas e 4 colunas, cujos tipos de dados podemos observar a seguir:

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id_cliente    10000 non-null  int64 
 1   nome_cliente  10000 non-null  object
 2   idade         10000 non-null  int64 
 3   endereco      10000 non-null  object
dtypes: int64(2), object(2)
memory usage: 312.6+ KB


Ainda, não temos valores faltantes.

In [15]:
df.isnull().sum()

id_cliente      0
nome_cliente    0
idade           0
endereco        0
dtype: int64

---

- Tabela ``fornecedores``;

In [17]:
query = """
SELECT * FROM fornecedores
"""

df = pd.read_sql_query(query, con = engine)
print(df.shape)
df

(10, 3)


,id_fornecedor,nome,contato
0,1,AstroSupply,contato@astrosupply.com
1,2,HorizonDistributors,contato@horizondistributors.com
2,3,EchoLogistics,contato@echologistics.com
3,4,CosmicConnections,contato@cosmicconnections.com
4,5,TerraTrade,contato@terratrade.com
5,6,PinnaclePartners,contato@pinnaclepartners.com
6,7,NebulaNetworks,contato@nebulanetworks.com
7,8,OceanicOrigins,contato@oceanicorigins.com
8,9,SummitSolutions,contato@summitsolutions.com
9,10,InfinityImports,contato@infinityimports.com


Essa tabela representa os fornecedores, identificados pela `PRIMARY KEY` em `id_fornecedor` e com informações de nome e contato para cada um deles. 

Pelo tamanho dessa, vemos que há 10 fornecedores presentes.

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_fornecedor  10 non-null     int64 
 1   nome           10 non-null     object
 2   contato        10 non-null     object
dtypes: int64(1), object(2)
memory usage: 372.0+ bytes


Também não possuimos dados faltantes nessa tabela.

In [19]:
df.isnull().sum()

id_fornecedor    0
nome             0
contato          0
dtype: int64

---

- Tabela ``marcas``;

In [22]:
query = """
SELECT * FROM marcas
"""

df = pd.read_sql_query(query, con = engine)
df.head()

,id_marca,nome
0,1,BluePeak
1,2,ZenithWave
2,3,SolarFlare
3,4,EchoBloom
4,5,CrystalCrest


---

- Tabela ``produtos``;

In [14]:
query = """
SELECT * FROM produtos
"""

df = pd.read_sql_query(query, con = engine)
df.head()

,id_produto,nome_produto,preco,categoria_id,marca_id,fornecedor_id,data_estoque,status
0,1,Bola de Futebol,24.48,4,6,10,2020-12-27,Vendido
1,2,Chocolate,1351.17,4,4,2,2021-02-13,Vendido
2,3,Celular,1351.88,4,3,4,2021-03-08,Vendido
3,4,Chocolate,662.65,5,1,10,2023-05-31,Vendido
4,5,Celular,1705.50,1,4,4,2021-01-19,Vendido


---

- Tabela ``vendas``;

In [15]:
query = """
SELECT * FROM vendas
"""

df = pd.read_sql_query(query, con = engine)
df.head()

,id_venda,data_venda,total_venda,cliente_id
0,1,2021-05-01,167.21,8112
1,2,2023-01-12,1315.33,76
2,3,2023-06-09,3044.65,1285
3,4,2021-01-31,4576.87,3897
4,5,2022-02-09,3429.25,2749


---

- Tabela ``itens_venda``;

In [16]:
query = """
SELECT * FROM itens_venda
"""

df = pd.read_sql_query(query, con = engine)
df.head()

,venda_id,produto_id
0,1,1510
1,2,1762
2,3,6889
3,3,5477
4,3,2181


---